In [2]:
from pgtools import panaroo_parser
from pgtools.gff_parser import parse_gff, parse_GFFs_dir, Pangenome_Gffs
from pgtools.gfa_parser import parse_gfa1
from pgtools.maf_parser import parse_maf, MAF
from pgtools.utils import intersection_len, contains
from pgtools.pangenome import Pangenome
import os
import argparse
from Bio.Seq import Seq

In [3]:
maf_dir = "/home/pampuch/studia/magisterka/test_data/klebsiella_subset_old/panaroo_new.maf"
gff_dir = "/home/pampuch/studia/magisterka/test_data/klebsiella_subset_old/gff/"

In [4]:
pg_maf = parse_maf(maf_dir)
pg_maf = MAF(list(pg_maf.seq_collections)[:30])

In [5]:
pg_maf.detect_soft_core()

In [6]:
for seq_coll in pg_maf.seq_collections:
    print(seq_coll.soft_core)

False
False
True
False
True
False
False
True
False
False
True
False
False
True
False
True
False
False
False
False
True
True
False
True
False
True
False
True
True
False


In [7]:
pg_maf.map_to_gff(gff_dir)

In [8]:
def annotations_to_csv(pg, gff_dir, csv_name = "annotations_summary.csv"):
    res_csv = open(csv_name, "w")
    res_csv.write("cluster id,cluster size,core status,seq name,mapped annotations\n")
    res_csv.flush()
    pg.detect_soft_core()
    pg.map_to_gff(gff_dir)
    for seq_coll in pg.seq_collections:
        id = seq_coll.id
        clust_size = len(seq_coll)
        core_status = seq_coll.soft_core
        for seq in seq_coll.sequences:
            # annotation len is also an iimportant aspect, but can be easily retrived from gff file
            # print(seq.seq_name, [ann.annotation_id for ann in seq.mapped_annotations], seq_coll.soft_core)
            annots = ";".join([ann.annotation_id for ann in seq.mapped_annotations])
            res_csv.write(f"{id},{clust_size},{core_status},{seq.seq_name},{annots}\n")
            res_csv.flush()
    res_csv.close()

In [9]:
annotations_to_csv(pg_maf, gff_dir)

In [10]:
for seq_coll in pg_maf.seq_collections:
    print(seq_coll.id)
    for seq in seq_coll.sequences:
        print(seq.seq_name, [ann.annotation_id for ann in seq.mapped_annotations], seq_coll.soft_core)

1812
5193_8_2.contig00013 [] False
5299_7_4.contig00074 ['MOOPCBGB_03461', 'MOOPCBGB_03461'] False
5197_2_1.contig00017 [] False
5197_7_4.contig00014 ['IMGDPOBN_02460', 'IMGDPOBN_02460'] False
5150_1_3.contig00008 ['AOMCCKAJ_02145', 'AOMCCKAJ_02145'] False
5197_7_5.contig00018 ['EHMCBADO_03703', 'EHMCBADO_03703'] False
5235_1_4.contig00241 ['MGBABOJF_03034', 'MGBABOJF_03034'] False
5150_2_2.contig00033 ['ILAAFBJJ_04801', 'ILAAFBJJ_04801'] False
5235_2_1.contig00014 [] False
5150_3_5.contig00049 ['JGFEPBDE_04468', 'JGFEPBDE_04468'] False
5235_5_4.contig00002 ['LLOJMOCI_00102', 'LLOJMOCI_00102'] False
5151_2_6.contig00014 ['NALFBGJE_02189', 'NALFBGJE_02189'] False
5235_6_12.contig00109 ['LIDJBJFH_02607', 'LIDJBJFH_02607'] False
5151_6_6.contig00019 [] False
5235_6_6.contig00049 ['LNOJILEH_01343', 'LNOJILEH_01343'] False
5193_1_5.contig00033 [] False
5299_1_3.contig00042 [] False
5193_2_6.contig00017 ['FGIHKDBE_02712', 'FGIHKDBE_02712'] False
3084
5235_1_4.contig00334 [] False
5197_7_5.co

## Extracting core from Panaroo

In [11]:
panaroo_dir = "/home/pampuch/studia/magisterka/test_data/klebsiella_subset_old/panaroo_out/"
panaroo_model = panaroo_parser.parse_panaroo_output(panaroo_dir, gff_dir)

In [12]:
def panaroo_annotations_to_csv(pg, gff_dir, csv_name = "panaroo_annotations_summary.csv"):
    res_csv = open(csv_name, "w")
    res_csv.write("cluster name,cluster size,core status,seq name,CDS\n")
    res_csv.flush()
    pg.detect_soft_core()
    # pg.map_to_gff(gff_dir)
    for seq_coll in pg.seq_collections:
        id = seq_coll.cluster_name
        clust_size = len(seq_coll)
        core_status = seq_coll.soft_core
        for seq in seq_coll.sequences:
            # annotation len is also an iimportant aspect, but can be easily retrived from gff file
            # print(seq.seq_name, [ann.annotation_id for ann in seq.mapped_annotations], seq_coll.soft_core)
            annots = ";".join([ann_id for ann_id in seq.annotation_ids])
            res_csv.write(f"{id},{clust_size},{core_status},{seq.seq_name},{annots}\n")
            res_csv.flush()
    res_csv.close()

In [13]:
panaroo_annotations_to_csv(panaroo_model, gff_dir)

In [14]:
summary_dir = "/home/pampuch/studia/magisterka/gffs_summary/"

## Klebsiella

In [16]:
import pandas as pd

In [15]:
common_contigs = os.path.join(summary_dir, "common_contig_pairs/klebsiella_subset/contig_pairs.csv")

In [38]:
df_pairs = pd.read_csv(common_contigs)

In [18]:
df_pairs

,contig1,contig2
0,5197_2_1.contig00015,5235_2_1.contig00493
1,5150_2_2.contig00001,5235_6_12.contig00341
2,5151_6_6.contig00062,5193_8_2.contig00060
3,5197_7_5.contig00010,5235_2_1.contig00112
4,5235_2_1.contig00505,5299_1_3.contig00053
...,...,...
100712,5299_1_3.contig00124,5299_7_4.contig00031
100713,5193_1_5.contig00050,5299_1_3.contig00107
100714,5150_3_5.contig00056,5151_6_6.contig00024
100715,5193_1_5.contig00005,5235_1_4.contig00571


In [36]:
df_lens = pd.read_csv(os.path.join(summary_dir, "klebsiella_subset.csv"))

In [37]:
df_lens["sequence"] = df_lens["genome"] + "." + df_lens["contig"]
df_lens

,genome,contig,contig length,no. of annotations on contig,sequence
0,5235_5_12,contig00001,64326,62,5235_5_12.contig00001
1,5235_5_12,contig00002,60612,63,5235_5_12.contig00002
2,5235_5_12,contig00003,53131,47,5235_5_12.contig00003
3,5235_5_12,contig00004,42928,47,5235_5_12.contig00004
4,5235_5_12,contig00005,40521,32,5235_5_12.contig00005
...,...,...,...,...,...
15603,5235_6_6,contig01171,56,0,5235_6_6.contig01171
15604,5235_6_6,contig01172,56,0,5235_6_6.contig01172
15605,5235_6_6,contig01173,56,0,5235_6_6.contig01173
15606,5235_6_6,contig01174,56,0,5235_6_6.contig01174


In [39]:
df_lens = df_lens.set_index("sequence")

In [27]:
df_lens

,genome,contig,contig length,no. of annotations on contig
sequence,,,,
5235_5_12.contig00001,5235_5_12,contig00001,64326,62
5235_5_12.contig00002,5235_5_12,contig00002,60612,63
5235_5_12.contig00003,5235_5_12,contig00003,53131,47
5235_5_12.contig00004,5235_5_12,contig00004,42928,47
5235_5_12.contig00005,5235_5_12,contig00005,40521,32
...,...,...,...,...
5235_6_6.contig01171,5235_6_6,contig01171,56,0
5235_6_6.contig01172,5235_6_6,contig01172,56,0
5235_6_6.contig01173,5235_6_6,contig01173,56,0


In [40]:
df_pairs["len1"] = ""
df_pairs["len2"] = ""
for row in df_pairs.iterrows():
    # print(df_lens.at[row[1]["contig1"], "contig length"])
    # print(row[0])
    df_pairs.at[row[0],"len1"] = df_lens.at[row[1]["contig1"], "contig length"]
    df_pairs.at[row[0],"len2"] = df_lens.at[row[1]["contig2"], "contig length"]


In [45]:
df_pairs.sort_values(["len1","len2"], ascending = [False, False])

,contig1,contig2,len1,len2
60682,5150_1_3.contig00001,5151_6_6.contig00002,520083,382901
16559,5150_1_3.contig00001,5197_7_5.contig00004,520083,339044
43442,5150_1_3.contig00001,5150_3_5.contig00001,520083,309492
79207,5150_1_3.contig00001,5151_2_6.contig00002,520083,256313
3411,5150_1_3.contig00001,5197_7_5.contig00009,520083,216796
...,...,...,...,...
46582,5150_3_5.contig00243,5299_1_3.contig00399,221,623
74187,5150_3_5.contig00243,5193_1_5.contig00196,221,466
34397,5150_3_5.contig00243,5235_6_11.contig00700,221,380
42910,5150_3_5.contig00243,5197_2_1.contig00206,221,310


In [46]:
res_dir = "/home/pampuch/studia/magisterka/gffs_summary/common_with_lens/"

In [47]:
df_pairs.to_csv(os.path.join(res_dir, 'klebsiella_subset.csv'))

In [53]:
from tqdm import tqdm

In [58]:
for dataset in os.listdir(os.path.join(summary_dir, "common_contig_pairs")):
    print(dataset)
    common_contigs = os.path.join(summary_dir, f"common_contig_pairs/{dataset}/contig_pairs.csv")
    df_pairs = pd.read_csv(common_contigs)
    df_lens = pd.read_csv(os.path.join(summary_dir, f"{dataset}.csv"))
    df_lens["sequence"] = df_lens["genome"] + "." + df_lens["contig"]
    df_lens = df_lens.set_index("sequence")
    df_pairs["len1"] = ""
    df_pairs["len2"] = ""
    for row in tqdm(df_pairs.iterrows()):
        # print(df_lens.at[row[1]["contig1"], "contig length"])
        # print(row[0])
        df_pairs.at[row[0],"len1"] = df_lens.at[row[1]["contig1"], "contig length"]
        df_pairs.at[row[0],"len2"] = df_lens.at[row[1]["contig2"], "contig length"]
    df_pairs = df_pairs.sort_values(["len1","len2"], ascending = [False, False])
    df_pairs.to_csv(os.path.join(res_dir, f"{dataset}.csv"))
    

    

sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented


2015684it [02:21, 14203.30it/s]


sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated


359097it [00:24, 14529.44it/s]


klebsiella_subset


...

321504it [00:22, 14490.84it/s]


sim_gr_1e-11_lr_1e-12_mu_1e-14_rep1


91331it [00:06, 14857.23it/s]


GPSC_subset


15880it [00:01, 15016.22it/s]


In [59]:
df = pd.read_csv(os.path.join(res_dir, "GPSC_subset.csv"))

In [60]:
df

,Unnamed: 0,contig1,contig2,len1,len2
0,3596,22841_3_158.velvet.ERS1699832_SC_contig000001,23164_8_137.velvet._SC_contig000001,413792,385873
1,15475,22841_3_158.velvet.ERS1699832_SC_contig000001,23164_8_137.velvet._SC_contig000002,413792,306798
2,3244,22841_3_158.velvet.ERS1699832_SC_contig000001,22841_3_7.velvet.ERS1699673_SC_contig000002,413792,270780
3,14237,22841_3_158.velvet.ERS1699832_SC_contig000001,22841_3_20.velvet.ERS1699686_SC_contig000001,413792,270333
4,7266,22841_3_158.velvet.ERS1699832_SC_contig000001,23164_8_98.velvet._SC_contig000001,413792,269971
...,...,...,...,...,...
15875,2690,15841_4_17.velvet.ERS639860_SC_contig000232,15841_4_38.velvet.ERS639881_SC_contig000001,301,275841
15876,3329,15841_4_17.velvet.ERS639860_SC_contig000232,17870_7_102.velvet.ERS725745_SC_contig000002,301,221843
15877,8443,15841_4_17.velvet.ERS639860_SC_contig000232,17870_7_11.velvet.ERS725493_SC_contig000004,301,178059
15878,9379,15841_4_17.velvet.ERS639860_SC_contig000232,17794_8_81.velvet.ERS719881_SC_contig000013,301,80577


In [61]:
df[:10]

,Unnamed: 0,contig1,contig2,len1,len2
0,3596,22841_3_158.velvet.ERS1699832_SC_contig000001,23164_8_137.velvet._SC_contig000001,413792,385873
1,15475,22841_3_158.velvet.ERS1699832_SC_contig000001,23164_8_137.velvet._SC_contig000002,413792,306798
2,3244,22841_3_158.velvet.ERS1699832_SC_contig000001,22841_3_7.velvet.ERS1699673_SC_contig000002,413792,270780
3,14237,22841_3_158.velvet.ERS1699832_SC_contig000001,22841_3_20.velvet.ERS1699686_SC_contig000001,413792,270333
4,7266,22841_3_158.velvet.ERS1699832_SC_contig000001,23164_8_98.velvet._SC_contig000001,413792,269971
5,9914,22841_3_158.velvet.ERS1699832_SC_contig000001,22841_3_20.velvet.ERS1699686_SC_contig000002,413792,243230
6,4057,22841_3_158.velvet.ERS1699832_SC_contig000001,23164_8_98.velvet._SC_contig000002,413792,243170
7,3848,22841_3_158.velvet.ERS1699832_SC_contig000001,22841_3_7.velvet.ERS1699673_SC_contig000004,413792,187194
8,13266,22841_3_158.velvet.ERS1699832_SC_contig000001,22841_3_20.velvet.ERS1699686_SC_contig000003,413792,187152
9,9157,22841_3_158.velvet.ERS1699832_SC_contig000001,23164_8_98.velvet._SC_contig000003,413792,186961


In [62]:
top_dir = "/home/pampuch/studia/magisterka/gffs_summary/top_contigs"

In [64]:
for dir in os.listdir(res_dir):
    dataset = dir.split(".")[0]
    df = pd.read_csv(os.path.join(res_dir, dir))
    df[:20].to_csv(os.path.join(top_dir, dir))

In [67]:
df_small = pd.read_csv(os.path.join(top_dir, "klebsiella_subset.csv"))

In [71]:
df_small[["contig1", "contig2"]].to_numpy()

array([['5150_1_3.contig00001', '5151_6_6.contig00002'],
       ['5150_1_3.contig00001', '5197_7_5.contig00004'],
       ['5150_1_3.contig00001', '5150_3_5.contig00001'],
       ['5150_1_3.contig00001', '5151_2_6.contig00002'],
       ['5150_1_3.contig00001', '5197_7_5.contig00009'],
       ['5150_1_3.contig00001', '5150_1_3.contig00006'],
       ['5150_1_3.contig00001', '5151_6_6.contig00005'],
       ['5150_1_3.contig00001', '5197_2_1.contig00004'],
       ['5150_1_3.contig00001', '5151_6_6.contig00007'],
       ['5150_1_3.contig00001', '5193_2_6.contig00006'],
       ['5150_1_3.contig00001', '5193_2_6.contig00007'],
       ['5150_1_3.contig00001', '5150_2_2.contig00010'],
       ['5150_1_3.contig00001', '5197_2_1.contig00009'],
       ['5150_1_3.contig00001', '5197_7_4.contig00008'],
       ['5150_1_3.contig00001', '5151_6_6.contig00010'],
       ['5150_1_3.contig00001', '5150_2_2.contig00015'],
       ['5150_1_3.contig00001', '5193_2_6.contig00010'],
       ['5150_1_3.contig00001',